# 1.预处理数据

## 1.1 对点云数据进行处理 - 滤除环视范围外过远的点

1. 解析 images.txt，计算所有相机在世界坐标系下的光心（位姿）。  
2.  使用 Open3D 加载 .pcd 点云，并基于与最近相机光心的距离进行滤除。  
3.  输出相机光心的范围（坐标轴最小/最大值）以及被滤除的点云数量。  
4.  将保留的点云按 COLMAP points3D.txt 格式保存，并额外生成一个 PLY 文件，其中红色标记被删除的点，原色保留未删除点。

#### ipynb内部函数 -- 未进行地面与boundingbox对齐

In [ ]:
import numpy as np
import open3d as o3d
from scipy.spatial.transform import Rotation as R

def filter_point_cloud(images_txt, pcd_path, output_points_txt, output_points_ply, output_ply_path):
    """
    根据相机位姿过滤点云，并保存结果。

    Args:
        images_txt (str): 包含相机位姿的 images.txt 文件路径。
        pcd_path (str): 点云文件路径。
        output_points_txt (str): 保存过滤后点云的 points3D.txt 文件路径。
        output_ply_path (str): 保存带颜色标记的 PLY 文件路径。
    """
    def load_camera_centers(images_txt):
        centers = []
        with open(images_txt, 'r') as f:
            for line in f:
                if line.startswith('#') or len(line.strip()) == 0:
                    continue
                elems = line.strip().split()
                if len(elems) < 10:
                    continue
                qw, qx, qy, qz = map(float, elems[1:5])
                tx, ty, tz = map(float, elems[5:8])
                rot = R.from_quat([qx, qy, qz, qw]).as_matrix()
                center = -rot.T @ np.array([tx, ty, tz])
                centers.append(center)
        return np.array(centers)

    # 1. 加载相机中心
    camera_centers = load_camera_centers(images_txt)

    # 2. 计算平均中心和最大偏移
    center_mean = camera_centers.mean(axis=0)
    offsets = np.abs(camera_centers - center_mean)
    max_offset = offsets.max(axis=0)

    # 3. 加载点云
    pcd = o3d.io.read_point_cloud(pcd_path)
    points = np.asarray(pcd.points)
    colors = np.asarray(pcd.colors) if pcd.has_colors() else np.ones_like(points)

    # 4. 构建包围盒并裁剪点云
    min_bound = center_mean - max_offset - 15
    max_bound = center_mean + max_offset + 15
    aabb = o3d.geometry.AxisAlignedBoundingBox(min_bound, max_bound)
    pcd_cropped = pcd.crop(aabb)

    # 5. 统计信息
    total_points = len(points)
    kept_points = np.asarray(pcd_cropped.points)
    kept_colors = np.asarray(pcd_cropped.colors) if pcd_cropped.has_colors() else np.ones_like(kept_points)
    removed_mask = np.ones(total_points, dtype=bool)
    kept_indices = aabb.get_point_indices_within_bounding_box(pcd.points)
    removed_mask[kept_indices] = False
    removed_points = points[removed_mask]
    removed_colors = colors[removed_mask]

    print("相机中心范围：")
    print(f"  X: [{min_bound[0]:.3f}, {max_bound[0]:.3f}]")
    print(f"  Y: [{min_bound[1]:.3f}, {max_bound[1]:.3f}]")
    print(f"  Z: [{min_bound[2]:.3f}, {max_bound[2]:.3f}]")
    print(f"总点数：{total_points}")
    print(f"保留点数：{len(kept_points)}")
    print(f"删除点数：{len(removed_points)}")

    # 6. 保存 points3D.txt (速度太慢，直接使用 ply )
    with open(output_points_txt, "w") as f:
        f.write("# 3D point list with one line of data per point:\n")
        f.write("# POINT3D_ID, X, Y, Z, R, G, B, ERROR, TRACK[]\n")
        for idx, (pt, col) in enumerate(zip(kept_points, kept_colors), start=1):
            r, g, b = (col * 255).astype(int)
            f.write(f"{idx} {pt[0]:.6f} {pt[1]:.6f} {pt[2]:.6f} {r} {g} {b} 0\n")
    # 保存为 ply 文件
    pcd_kept = o3d.geometry.PointCloud()
    pcd_kept.points = o3d.utility.Vector3dVector(kept_points)
    pcd_kept.colors = o3d.utility.Vector3dVector(kept_colors)
    o3d.io.write_point_cloud(output_points_ply, pcd_kept)
    print("保留的点云已保存为 {output_points_ply}")
    
    # 7. 保存带颜色的 PLY 文件
    all_points = np.vstack((kept_points, removed_points))
    removed_colors_red = np.tile([1.0, 0.0, 0.0], (len(removed_points), 1))
    all_colors = np.vstack((kept_colors, removed_colors_red))
    pcd_all = o3d.geometry.PointCloud()
    pcd_all.points = o3d.utility.Vector3dVector(all_points)
    pcd_all.colors = o3d.utility.Vector3dVector(all_colors)
    o3d.io.write_point_cloud(output_ply_path, pcd_all)

In [14]:
import os
folder_path = '/home/qinllgroup/hongxiangyu/git_project/gaussian-splatting-xy/data/tree_01_livo2/livo2_results'

images_txt = os.path.join(folder_path, 'sparse/0/images.txt')
pcd_path = os.path.join(folder_path, 'pcd/all_raw_points.pcd')
output_points_ply = os.path.join(folder_path, 'pcd/points3D.ply')
vis_ply_path = os.path.join(folder_path, 'pcd/vis_filter.ply')
output_points_txt = os.path.join(folder_path, 'pcd/points3D.txt')
filter_point_cloud(images_txt,pcd_path,output_points_txt, output_points_ply,vis_ply_path)

相机中心范围：
  X: [-0.200, 28.270]
  Y: [-16.082, 1.699]
  Z: [-9.392, 0.714]
总点数：16404321
保留点数：10684495
删除点数：5719826
保留的点云已保存为 {output_points_ply}


#### 调用 crop_points.py 文件

In [ ]:
!python crop_points.py \
    --images_txt /home/qinllgroup/hongxiangyu/git_project/gaussian-splatting-xy/data/tree_01_livo2/livo2_results/sparse/0/images.txt \
    --pcd_path /home/qinllgroup/hongxiangyu/git_project/gaussian-splatting-xy/data/tree_01_livo2/livo2_results/pcd/all_raw_points.pcd \
    --output_points_txt /home/qinllgroup/hongxiangyu/git_project/gaussian-splatting-xy/data/tree_01_livo2/livo2_results/pcd/points3D.txt \
    --output_ply /home/qinllgroup/hongxiangyu/git_project/gaussian-splatting-xy/data/tree_01_livo2/livo2_results/pcd/filtered_colored.ply \
    --margin 10.0

总点数：16404321，保留：15254842，删除：1149479


## 1.2 利用体素化降采样

In [2]:
import os
from tools.points_utils import voxel_downsample_and_save

voxel_size = 0.05
folder_path = '/home/qinllgroup/hongxiangyu/git_project/gaussian-splatting-xy/data/tree_01_livo2/livo2_results'
input_ply_path = os.path.join(folder_path,'pcd/points3D.ply')
output_ply_path = os.path.join(folder_path,f'pcd/points3D_{voxel_size}.ply')
input_ply_path = '/home/qinllgroup/hongxiangyu/git_project/gaussian-splatting-xy/data/tree_01_debug/mini3/sparse/0/raw/all_raw_points.ply'
output_ply_path = f'/home/qinllgroup/hongxiangyu/git_project/gaussian-splatting-xy/data/tree_01_debug/mini3/sparse/0/raw/all_raw_points_{voxel_size}.ply'

voxel_downsample_and_save(voxel_size, input_ply_path, output_ply_path) # ply downsample to ply


raw points len : 16404321
downsample points len : 4671298
降采样后的点云已保存到 /home/qinllgroup/hongxiangyu/git_project/gaussian-splatting-xy/data/tree_01_debug/mini3/sparse/0/raw/all_raw_points_0.05.ply，体素大小：0.05


In [1]:
# pcd 2 ply
# 将原pcd点云，转换为ply点云
from tools.points_utils import pcd_2_ply
pcd_file = '/home/qinllgroup/hongxiangyu/git_project/gaussian-splatting-xy/data/tree_01_debug/mini3/sparse/0/raw/all_raw_points.pcd'
ply_file = '/home/qinllgroup/hongxiangyu/git_project/gaussian-splatting-xy/data/tree_01_debug/mini3/sparse/0/raw/all_raw_points.ply'

pcd_2_ply(pcd_file,ply_file)


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Converted /home/qinllgroup/hongxiangyu/git_project/gaussian-splatting-xy/data/tree_01_debug/mini3/sparse/0/raw/all_raw_points.pcd to /home/qinllgroup/hongxiangyu/git_project/gaussian-splatting-xy/data/tree_01_debug/mini3/sparse/0/raw/all_raw_points.ply


In [1]:
# pcd 2 txt
from tools.points_utils import pcd_2_colmap_txt
pcd_file = '/home/qinllgroup/hongxiangyu/git_project/gaussian-splatting-xy/data/tree_01_livo2/sparse/0/points3D_filter.ply'
txt_file = '/home/qinllgroup/hongxiangyu/git_project/gaussian-splatting-xy/data/tree_01_livo2/livo2_results/pcd/points3D_filter_white.txt'

pcd_2_colmap_txt(pcd_file, txt_file, is_white=True)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
正在写入COLMAP TXT文件: /home/qinllgroup/hongxiangyu/git_project/gaussian-splatting-xy/data/tree_01_livo2/livo2_results/pcd/points3D_filter_white.txt
Converted /home/qinllgroup/hongxiangyu/git_project/gaussian-splatting-xy/data/tree_01_livo2/sparse/0/points3D_filter.ply to Colmap TXT in: /home/qinllgroup/hongxiangyu/git_project/gaussian-splatting-xy/data/tree_01_livo2/livo2_results/pcd/points3D_filter_white.txt


选择一个合适的 points3D.ply 文件复制到 sparse/0 下

## 1.2 Resize for more images

In [3]:
import os
import cv2
from tqdm import tqdm

def resize_images(input_dir, output_dir, extensions):
    """
    读取输入文件夹中的所有图片，调整为1/2大小后保存到输出文件夹
    
    Args:
        input_dir: 输入图片文件夹路径
        output_dir: 输出图片文件夹路径
        extensions: 支持的图片扩展名列表
    """
    # 确保输出目录存在
    os.makedirs(output_dir, exist_ok=True)
    
    # 获取所有图片文件
    image_files = []
    for file in os.listdir(input_dir):
        if any(file.lower().endswith(ext) for ext in extensions):
            image_files.append(file)
    
    if not image_files:
        print(f"在 {input_dir} 中未找到支持的图片文件")
        return
    
    print(f"找到 {len(image_files)} 张图片")
    
    # 处理每张图片
    count = 0
    for file in tqdm(image_files, desc="处理中"):
        input_path = os.path.join(input_dir, file)
        output_path = os.path.join(output_dir, file)
        
        try:
            # 读取图片
            img = cv2.imread(input_path)
            if img is None:
                print(f"警告: 无法读取图片 {input_path}，跳过")
                continue
                
            # 获取原始尺寸
            height, width = img.shape[:2]
            
            # 计算新尺寸
            new_width = width // 2
            new_height = height // 2
            
            # 调整尺寸
            resized_img = cv2.resize(img, (new_width, new_height), interpolation=cv2.INTER_AREA)
            
            # 保存图片
            cv2.imwrite(output_path, resized_img)
            
            # 输出尺寸信息
            if count == 0:
                print(f"{file}: {width}x{height} -> {new_width}x{new_height}")
            count += 1
        except Exception as e:
            print(f"错误: 处理图片 {input_path} 时出错: {str(e)}")


In [ ]:
input_path = '/home/qinllgroup/hongxiangyu/git_project/gaussian-splatting-xy/data/tree_01/depth_maps'
output_path = '/home/qinllgroup/hongxiangyu/git_project/gaussian-splatting-xy/data/tree_01/depth_maps_2'
exts = ['jpg','jpeg','png']

resize_images(input_path, output_path, exts)
print("所有图片处理完成！")    

# 2. LIVO2和Colmap的重建对比实验

colmap无法恢复相机位姿；所以这里我们使用livo2恢复位姿后，用colmap 进行三角测量获取关键点
https://www.cnblogs.com/Todd-Qi/p/15080968.html

## 2.1 基于Livo2位姿进行稀疏重建

colmap无法恢复相机位姿；所以这里我们使用livo2恢复位姿后，用colmap 进行三角测量获取关键点
https://www.cnblogs.com/Todd-Qi/p/15080968.html

1. 准备来自Livo2的位姿和相机数据 cameras.txt, images.txt
    将内参(camera intrinsics) 放入cameras.txt， 外参(camera extrinsics)放入 images.txt , points3D.txt 为空  
    - images.txt 中全部 0.0 0.0 -1 删除;  
    - points3D.txt 内容清空;
    - cameras.txt 中的内参进行修改 (对输入图像全部进行了 resize 操作，因此需要修改相机内参，将fx, fy, cx, cy 都除以2)

2. 特征匹配与特征提取  
``` bash
        colmap feature_extractor \
            --database_path /path/to/project/database.db \  
            --image_path    /path/to/project/images
```
``` bash
        colmap exhaustive_matcher \
            --database_path /path/to/project/database.db
```

3. 三角化重建 (保存的点云和其他文件均为bin格式)
``` bash
        colmap point_triangulator \
            --database_path /path/to/project/database.db \
            --image_path    /path/to/project/images \
            --input_path    /path/to/sparse_model \
            --output_path   /path/to/triangulated_model

```

查看txt结果
``` bash
        colmap model_converter \
            --input_path 0 \
            --output_path 0_txt_from_livo2 \
            --output_type TXT
```

4. 稠密重建(optional)

# 3.训练

In [ ]:
# baseline raw gs for training
!CUDA_VISIBLE_DEVICES=1 python train.py \
    -s data/tree_01_livo2 \
    -m data/tree_01_livo2/outputs/3dgs_baseline
    
# render
!CUDA_VISIBLE_DEVICES=1 python render.py \
    -s data/tree_01_colmap \
    -m data/tree_01_colmap/outputs/3dgs_baseline    

In [ ]:

iss = [[1,5,4],[2,4,5]]
iss.sort(key=lambda x: [x[1],x[0]], reverse=True)
iss

[[2, 4, 5], [1, 5, 4]]

In [ ]:
class Solution(object):
    def merge(self, intervals):
        """
        :type intervals: List[List[int]]
        :rtype: List[List[int]]
        """
        if not intervals:
            return []
        # 先按区间的起始位置排序
        intervals.sort(key=lambda x: x[0])
        merged = [intervals[0]]
        for i in range(1, len(intervals)):
            # 如果当前区间与上一个区间重叠，则合并
            if intervals[i][0] <= merged[-1][1]:
                merged[-1][1] = max(merged[-1][1], intervals[i][1])
            else:
                merged.append(intervals[i])
        return merged